In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys, os , argparse

# Constant Parameters for CFP modeling
The following parameters are constants, representing the maximun, minium and most probable values sampled from our probabilistic distribution. If you re-run the **probabilistic sequence generation** in `cfp_calculation.ipynb`, you may obtain slightly different values.


# Read and clean the base dataset

In [2]:
processors = pd.read_excel('DataSetWithRelaventNodesArea.xlsx')
processors = processors.dropna(subset=['DieSizeValue', 'Die Size (mm^2)'], how='all')
processors = processors[(processors['DieSizeValue'] != 0) | (processors['Die Size (mm^2)'] != 0)]
processors = processors[processors['DieSizeValue'] != 'unknown']
processors = processors[processors['Die Size (mm^2)'] != 'unknown']
# Chiplet processors have different notations in die size value and die size
chiplet = processors[processors['DieSizeValue'] != processors['Die Size (mm^2)']]


processors = processors.drop(processors[processors['DieSizeValue'] != processors['Die Size (mm^2)']].index)
processors = processors[processors['TDP (W)'].notna()]
processors = processors[processors['TDP (W)'] != 'unknown']
processors = processors[processors['TDP (W)'] != '0']

## Following code block generates the probabilistic distribution of manufacturing parameters required by ECO-CHIP. 
## We then perform Monte-Carlo simulation using these generated samples to calculate the final CFP of each processor

In [ ]:
sample_size = 1000000
ci_model_mean = 522.936
ci_model_std = 19.826
ci_model = np.random.choice(np.arange(480.8486,546.7014,2.057901), size=sample_size, replace=True, p = [0.03125,0.03125,0.03125,0.03125,0.01041667,0.01041667,0.01041667,0.01041663,0.01041667,
                                                                                                  0.01041667,0.01041667,0.01041667,0.01041667,0.01041667,0.01041667,
                                                                                                  0.01041667,0.04166667,0.04166667,0.04166667,0.04166667,
                                                                                                  0.02083333,0.02083333,0.02083333,0.02083333,0.0625,0.0625,
                                                                                                  0.0625,0.0625,0.0625,0.0625,0.0625,0.0625])
gpa_mean=150
gpa_std=30

gpa_model = np.random.normal(gpa_mean, gpa_std, sample_size)
gpa_model = np.clip(gpa_model,50,300)



defective_density_bench =  {"7": 0.2, "10": 0.11, "14":0.09, "22": 0.08, "28":0.07, "65":0.05}
epa_bench =  {"7": 2.15, "10": 1.475, "14":1.2, "22": 1.2, "28":0.9}

epa_distribution = [0.092478422,0.09864365,
                    0.101726264,0.103575832,0.101726264,0.097410604,
                    0.091245376,0.08323058,0.073982737,0.061652281,
                    0.048088779,0.033908755,0.012330456,]

defective_distribution=[0.1125,0.1125,0.1125,0.1125,0.1125,0.05,0.05,0.05,0.05,0.05,
    0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,
    0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,]

def distribution_modeling(bench, p, stride = 25, d1 = None, d2 = None):
    d1 = 0.11 - 0.095
    d2 = 0.42 - 0.11
    defect_distribution = dict()
    for node in bench:
        defect_bench = defective_density_bench[node]
        start = defect_bench - d1
        end = defect_bench + d2
        step = (end - start) / stride
        defect_distribution[node] = np.random.choice(np.arange(start, end, step), size=sample_size, replace=True, p=p)

    return defect_distribution

def highest_probability(result):
    unique_values, counts = np.unique(result, return_counts=True)
    probabilities = counts / len(result)  # Convert frequency models to probabilities
    # Find the maximum probability and its corresponding value
    max_prob_index = np.argmax(probabilities)
    max_prob_value = unique_values[max_prob_index]
    max_prob = probabilities[max_prob_index]

    return max_prob_value, max_prob

In [8]:
sys.path.append(os.path.join(os.getcwd(), "../eco_chip_enhanced"))

from eco_chip_func import eco_chip
for idx, proc in processors.iterrows():

    # node = int(proc['Process Size (nm)'])
    node = int(proc['Process Size (nm)'])
    power = int(proc['TDP (W)'])
    area = int(proc['DieSizeValue'])

    defect = defect_density_min[str(node)]
    epa_val = epa_min[str(node)]
    gpa_val =gpa_min
    ci_val = ci_min

    args = argparse.Namespace(design_dir='testcases/CFP_survey/', 
                                chip_area=area, 
                                node=node, 
                                chip_power=power, 
                                defect_density=defect,
                                num_lifetime=None, 
                                gpa = gpa_val, 
                                epa = epa_val,
                                tech_scaling_path='../eco_chip_enhanced/',
                                carbon_intensity=ci_val)    
    c_des, c_mfg, c_ope, c_tot = eco_chip(args)
    min_emb = float(c_des) + float(c_mfg)
    min_ope = c_ope
    min_tot = min_emb + float(c_ope)
    processors.at[idx, 'Emb Crabon Min'] = min_emb
    processors.at[idx, 'Ope Carbon Min'] = min_ope
    processors.at[idx, 'Tot Carbon Min'] = min_tot
    processors.at[idx, 'ECFPA Min'] = min_emb / area


    defect = defect_density_max[str(node)]
    epa_val = epa_max[str(node)]
    gpa_val =gpa_max
    ci_val = ci_max

    args = argparse.Namespace(design_dir='testcases/CFP_survey/', 
                                chip_area=area, 
                                node=node, 
                                chip_power=power, 
                                defect_density=defect,
                                num_lifetime=None, 
                                gpa = gpa_val, 
                                epa = epa_val,
                                tech_scaling_path='../eco_chip_enhanced/',
                                carbon_intensity=ci_val)    
    c_des, c_mfg, c_ope, c_tot = eco_chip(args)
    max_emb = float(c_des) + float(c_mfg)
    max_ope = c_ope
    max_tot = max_emb + float(c_ope)

    processors.at[idx, 'Emb Crabon Max'] = max_emb
    processors.at[idx, 'Ope Carbon Max'] = max_ope
    processors.at[idx, 'Tot Carbon Max'] = max_tot
    processors.at[idx, 'ECFPA Max'] = max_emb / area

    defect = defect_density[str(node)]
    epa_val = epa[str(node)]
    gpa_val =gpa
    ci_val = ci

    args = argparse.Namespace(design_dir='testcases/CFP_survey/', 
                                    chip_area=area, 
                                    node=node, 
                                    chip_power=power, 
                                    defect_density=defect,
                                    num_lifetime=None, 
                                    gpa = gpa_val, 
                                    epa = epa_val,
                                    tech_scaling_path='../eco_chip_enhanced/',
                                    carbon_intensity=ci_val)    

    c_des, c_mfg, c_ope, c_tot = eco_chip(args)
    emb = float(c_des) + float(c_mfg)
    ope = c_ope
    tot = emb + float(c_ope)


    processors.at[idx, 'Emb Crabon Most Probable'] = emb
    processors.at[idx, 'Ope Carbon Most Probable'] = ope
    processors.at[idx, 'Tot Carbon Most Probable'] = tot
    processors.at[idx, 'ECFPA Most Probable'] = emb / area

print("Most Probable CFP value calculation Finished")

Most Probable CFP value calculation Finished


# Combine Chiplet and processed processors to CSV

In [10]:
processors
chiplet['Emb Crabon Min'] = 'Insuficient Data'
chiplet['Ope Carbon Min'] = 'Insuficient Data'
chiplet['Tot Carbon Min'] = 'Insuficient Data'
chiplet['ECFPA Min'] = 'Insuficient Data'

chiplet['Emb Crabon Max'] = 'Insuficient Data'
chiplet['Ope Carbon Max'] = 'Insuficient Data'  
chiplet['Tot Carbon Max'] = 'Insuficient Data'
chiplet['ECFPA Max'] = 'Insuficient Data'

chiplet['Emb Crabon Most Probable'] = 'Insuficient Data'
chiplet['Ope Carbon Most Probable'] = 'Insuficient Data'
chiplet['Tot Carbon Most Probable'] = 'Insuficient Data'
chiplet['ECFPA Most Probable'] = 'Insuficient Data'

combined_df = pd.concat([chiplet, processors], ignore_index=True)
combined_df.to_csv('../dataset/CarbonSet.csv', index=False)
